## HW4 - Zhamilya Saparova


In [139]:
pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [140]:
pip install ir_measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [141]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import pandas as pd
import numpy as np
import ir_measures
from ir_measures import *

In [142]:
es = Elasticsearch("http://localhost:9200")
ELASTIC_PASSWORD = 'OsTuAI29lih200BZT70Pp4Sy'

CLOUD_ID = 'ir_hw4:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRiYzA3ZDRkMGRhYjc0OGNmOGQyZGE0ZjEzZDYyMTY5OCQ4ZDEyYTI0ZGU5Yzc0ZmI4YTViMmFlZWU4M2M5NDcxMQ=='
es = Elasticsearch(cloud_id = CLOUD_ID, basic_auth=('elastic', ELASTIC_PASSWORD))

print("Connection to ES Server successful")

Connection to ES Server successful


In [143]:
es.indices.delete(index='hello')

ObjectApiResponse({'acknowledged': True})

In [144]:
es.indices.create(index='hello')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hello'})

In [145]:
settings_final = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'white':{
                        'type': 'text',
                        'analyzer': 'white_analyzer'
                    }
                }
            }
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                        'snowball_eng'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
            'filter': {
                'snowball_eng':{
                    'type': 'snowball',
                    'language': 'english'
                }
            }
        }
    }
    
}

In [146]:
settings_final_nostem = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'white':{
                        'type': 'text',
                        'analyzer': 'white_analyzer'
                    }
                }
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
        }
    }
}

In [147]:
def recreate_index():
    es.indices.delete(index='hello')
    es.indices.create(index='hello', body = settings_final)

In [148]:
def recreate_index_nostem():
    es.indices.delete(index='hello')
    es.indices.create(index='hello', body = settings_final_nostem)

In [149]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='hello', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [150]:
# recreate_index()

In [151]:
recreate_index_nostem()

<ipython-input-148-8cc1ef7b44ce>:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='hello', body = settings_final_nostem)


## INDEX DOCUMENTS

In [152]:
doc_df = pd.read_csv('/content/documents.csv')
doc = doc_df.to_dict('records')

In [153]:
type(doc), len(doc), type(doc_df), doc_df.shape

(list, 369721, pandas.core.frame.DataFrame, (369721, 2))

In [154]:
doc_df.head()

,id_right,text_right
0,1781133,it was used in landing craft during world war ...
1,2426736,after rejecting an offer from cambridge univer...
2,2224122,mat zan coached kuala lumpur fa in 1999 and wo...
3,219642,a barcode is a machine readable optical label ...
4,1728654,since the subordination of the monarchy under ...


In [155]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [156]:
def es_actions_generator():
    for doc_id in doc:
        yield create_es_action('hello', doc_id['id_right'], doc_id)

In [167]:
# ADDING DOCS TO THE INDEX
%%time 
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)

CPU times: user 1min 18s, sys: 710 ms, total: 1min 19s
Wall time: 2min 4s


## SEARCH


In [168]:
def search(query, *args):
    pretty_print_result(es.search(index='hello', body=query, size=20), args)
                        
def pretty_print_result(search_result, fields=[]):
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='hello', id=doc_id)['_source']

In [169]:
%%time 

query = {
    'query': {
        'match_all': {}
    }
}
search(query)

<ipython-input-168-fae58037f7d9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='hello', body=query, size=20), args)


Total documents: 10000
Doc 1781133, score is 1.0
Doc 2426736, score is 1.0
Doc 2224122, score is 1.0
Doc 219642, score is 1.0
Doc 1728654, score is 1.0
Doc 1889917, score is 1.0
Doc 1518473, score is 1.0
Doc 1459418, score is 1.0
Doc 1102956, score is 1.0
Doc 931408, score is 1.0
Doc 1043905, score is 1.0
Doc 488327, score is 1.0
Doc 1069841, score is 1.0
Doc 1601336, score is 1.0
Doc 1415191, score is 1.0
Doc 1531137, score is 1.0
Doc 912052, score is 1.0
Doc 935016, score is 1.0
Doc 1671729, score is 1.0
Doc 1946935, score is 1.0
CPU times: user 190 ms, sys: 9.1 ms, total: 199 ms
Wall time: 7.89 s


In [170]:
%%time

query = {
    'query': {
        'bool': {
            'must': {
                'match': {
                    'text_right': 'private boats'
                }
            }
        }
    }
}

search(query)

<ipython-input-168-fae58037f7d9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='hello', body=query, size=20), args)


Total documents: 10000
Doc 2355214, score is 13.976819
Doc 644717, score is 13.686264
Doc 1596321, score is 13.1172695
Doc 2412578, score is 12.66897
Doc 352785, score is 12.558475
Doc 1272069, score is 12.242398
Doc 1226233, score is 12.007725
Doc 1856656, score is 12.007725
Doc 880419, score is 12.007725
Doc 919902, score is 12.007725
Doc 1879751, score is 12.007725
Doc 1110284, score is 11.860317
Doc 1329021, score is 10.934549
Doc 2029858, score is 10.934549
Doc 1964437, score is 10.934549
Doc 1714910, score is 10.934549
Doc 551366, score is 10.934549
Doc 885487, score is 10.934549
Doc 2075473, score is 10.934549
Doc 1185710, score is 10.884844
CPU times: user 238 ms, sys: 8.86 ms, total: 247 ms
Wall time: 8.73 s


In [171]:
pip install -U sentence-transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [172]:
import datetime
import time
import torch
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('multi-qa-minilm-l6-cos-v1')

q = pd.read_csv('/content/queries.csv')
q = q.to_dict('records')
runs = dict()

for i in range(len(q)):
  st = time.time()
  queries = dict()
  query = {
        'query': {
            'bool': {
                'must': {
                    'match': {
                        'text_right': q[i]['text_left']
                    }
                }
            }
        }
    }
  search_results = es.search(index = 'hello', body = query, size = 20)
 
  corpus = []
  idx = dict()

  if len(search_results['hits']['hits']) != 0:

    for j in range(len(search_results['hits']['hits'])):
      corp = search_results['hits']['hits'][j]['_source']['text_right']
      idx[j] = search_results['hits']['hits'][j]['_source']['id_right']
      corpus.append(corp)
    
    top_k = min(20, len(corpus))
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    query_embedding = embedder.encode(q[i]['text_left'], convert_to_tensor = True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    for scoree, idxx in zip(top_results[0], top_results[1]):
      doc_id = str(idx[int(idxx)])
      score = float(scoree)
      query_id = i
      queries[doc_id] = score
    
    runs[str(q[i]['id_left'])] = queries

    et = time.time()
    elapsed_time = et - st
    print(query_id, '\t', elapsed_time, '\t', queries.keys())


<ipython-input-172-af43fef2c434>:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es.search(index = 'hello', body = query, size = 20)


0 	 3.876194715499878 	 dict_keys(['158491', '635537', '607552', '1880296', '1957435', '360918', '625257', '742912', '663828', '1774491', '2411344', '1093529', '685181', '1956922', '589549', '2261272', '945068', '1158969', '1170039', '1180246'])
1 	 4.229133605957031 	 dict_keys(['5728', '283099', '98452', '2149047', '79940', '1890681', '592647', '849904', '501880', '608495', '645822', '375146', '1893219', '970793', '1750830', '2184293', '911906', '255972', '1597007', '1404651'])
2 	 3.4031498432159424 	 dict_keys(['1354456', '13554', '506304', '166298', '1333624', '2164978', '1354099', '2164385', '2206256', '1231923', '1355603', '1604564', '107818', '1343180', '2108477', '1355975', '1070', '1354754', '1212060', '1354324'])
3 	 6.750439643859863 	 dict_keys(['1378846', '961758', '671380', '1317839', '968448', '784701', '834604', '375161', '1423812', '1415478', '1505849', '1572828', '958943', '693088', '870643', '910449', '1560729', '1151820', '1205770', '484924'])
4 	 5.876078128814697

## PART 2 EVALUATION

In [173]:
#  Qrel(query_id='13540', doc_id='1276852', relevance=1, iteration='0'),

qrels = ir_measures.read_trec_qrels('/content/qrels')
qrels_ = dict()
qrels_new = dict()
for i in list(qrels):
  query_id = i[0]
  doc_id = i[1]
  relevance = i[2]
  qrels_[doc_id] = relevance
  qrels_new[query_id] = qrels_

In [174]:
scores = ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels_new, runs)
scores

{AP: 0.00045862085630015234,
 P@10: 0.2419999999999999,
 P@20: 0.1640000000000001}

In [175]:
# [ScoredDoc(query_id='158491', doc_id='625257', score=15.660703104969318),

bm25 = ir_measures.read_trec_run('/content/BM25.res')
bm25_ = dict()
creators_runs = dict()
for i in list(bm25):
  query_id = i[0]
  doc_id = i[1]
  score = i[2]
  bm25_[doc_id] = score
  creators_runs[query_id] = bm25_

In [176]:
scores = ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels_new, creators_runs)
scores

{AP: 0.010808749000016847, P@10: 0.3000000000000005, P@20: 0.3000000000000005}